In [146]:
import numpy as np
import pandas as pd
import os
import re
import sklearn
from sklearn import metrics

In [147]:
# Data source: https://www.kaggle.com/datasets/gauthamp10/google-playstore-apps
path = os.getcwd() + "/data/Google-Playstore.csv"
df = pd.read_csv(path)

In [148]:
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312944 entries, 0 to 2312943
Data columns (total 24 columns):
 #   Column             Dtype  
---  ------             -----  
 0   App Name           object 
 1   App Id             object 
 2   Category           object 
 3   Rating             float64
 4   Rating Count       float64
 5   Installs           object 
 6   Minimum Installs   float64
 7   Maximum Installs   int64  
 8   Free               bool   
 9   Price              float64
 10  Currency           object 
 11  Size               object 
 12  Minimum Android    object 
 13  Developer Id       object 
 14  Developer Website  object 
 15  Developer Email    object 
 16  Released           object 
 17  Last Updated       object 
 18  Content Rating     object 
 19  Privacy Policy     object 
 20  Ad Supported       bool   
 21  In App Purchases   bool   
 22  Editors Choice     bool   
 23  Scraped Time       object 
dtypes: bool(4), float64(4), int64(1), object(15)
memor

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,NaN,vnacrewit@gmail.com,"Aug 9, 2019","Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,"Sep 10, 2018","Oct 13, 2018",Everyone,NaN,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,"Feb 21, 2020","Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35
5,IMOCCI,com.imocci,Social,0.0,0.0,50+,50.0,89,True,0.0,...,http://www.imocci.com,info@imocci.com,"Dec 24, 2018","Dec 20, 2019",Teen,https://www.imocci.com/wp-content/uploads/2018...,False,True,False,2021-06-15 20:19:35
6,unlimited 4G data prank free app,getfreedata.superfatiza.unlimitedjiodataprank,Libraries & Demo,4.5,12.0,"1,000+",1000.0,2567,True,0.0,...,NaN,aitomgharfatimezzahra@gmail.com,"Sep 23, 2019","Sep 27, 2019",Everyone,https://sites.google.com/view/unlimited4gdatap...,True,False,False,2021-06-15 20:19:35
7,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2.0,39.0,500+,500.0,702,True,0.0,...,NaN,elementuser03@gmail.com,"Jun 21, 2019","Jun 21, 2019",Everyone,https://www.freeprivacypolicy.com/privacy/view...,False,False,False,2021-06-15 20:19:35
8,WhatsOpen,com.whatsopen.app,Communication,0.0,0.0,10+,10.0,18,True,0.0,...,http://yilvermolinah.com,yilver.mh1996@gmail.com,NaN,"Dec 07, 2018",Teen,http://elcafedelamanana.yilvermolinah.com/poli...,False,False,False,2021-06-15 20:19:35
9,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,820.0,"50,000+",50000.0,62433,True,0.0,...,https://trendyteme888-31139.web.app,trendyteme.888@gmail.com,"Sep 22, 2019","Oct 07, 2020",Everyone,http://bit.ly/EmojiThemePro,True,False,False,2021-06-15 20:19:35


In [149]:
df = df.drop(['Scraped Time', 'Developer Id', 'Installs', 'Currency', 'Minimum Installs',  #'Size',
'Developer Website', 'Developer Email', 'Privacy Policy', 'Minimum Android', 'Content Rating', 
'Editors Choice'], axis=1)

#This one app had only 1 maximum install but an incredible number of ratings.
df = df[df['App Id'] != 'com.google.android.apps.wellbeing']

In [150]:
cat_name = 'Super Category'
df[cat_name] = df['Category']
df[cat_name] = df[cat_name].replace(['Adventure', 'Racing', 'Puzzle', 'Entertainment', 'Arcade', 'Photography', 'Sports', 'Card', 'Trivia', 'Strategy', 'Action', 'Simulation', 'Casino', 'Comics'], 'Entertainment')
df[cat_name] = df[cat_name].replace(['Communication', 'Social', 'Events', 'Dating', 'Role Playing', 'Lifestyle', 'Personalization', 'Medical', 'Health & Fitness', 'Beauty', 'Parenting'], 'Personal & Social')
df[cat_name] = df[cat_name].replace(['Productivity', 'Books & Reference', 'Education', 'Business', 'Educational', 'Finance'], 'Development')
df[cat_name] = df[cat_name].replace(['Casual', 'Tools', 'Libraries & Demo', 'Maps & Navigation', 'Travel & Local', 'Food & Drink', 'Music', 'Auto & Vehicles','Shopping', 'Board', 'Music & Audio', 'News & Magazines', 'Art & Design', 'House & Home', 'Weather', 'Word', 'Video Players & Editors'], 'Other')
df[cat_name] = df[cat_name].astype('category')
df[cat_name].value_counts()

Other                736773
Development          668061
Personal & Social    460998
Entertainment        447111
Name: Super Category, dtype: int64

In [151]:
df['Rating Bin'] = pd.cut(df['Rating'],bins=[0,3.9,4.4,5], labels=['Bad App Yo', 'Moderate', 'Superb'])
# labels=['Bad App Yo', 'Moderate', 'Superb']

In [152]:
df['App Name'] = df['App Name'].apply(str)
df['Name Length'] = df['App Name'].apply(len)

In [153]:
#What kind of Size values exist, going by the text part of the string
SizeCats = df['Size'].apply(lambda x: re.findall('([\sA-Za-z]*$)', str(x))[0])
print(SizeCats.value_counts())

#Knowing there are three kinds of known sizes (kb, Mb, Gb) we convert these to Mb
#But because there are also missing values or OS dependent sizes, we store this categorically in an extra column
#(known sizes will be categorized as 'known')
def process_Size(value):
    value = str(value)
    conversion = {'k':0.001, 'M':1.0, 'G':1000.0}
    if value[-1] in conversion.keys():
        value = value.replace(',', '') #added because some 
        size = float(value[:-1]) * conversion[value[-1]]
        category = 'Known'
    else:
        size = None
        category = value
    return (size, category)


temp_df = pd.DataFrame(df['Size'].apply(lambda x: process_Size(x)).tolist(), columns=['Size', 'Size Cat'])
df['Size'] = temp_df['Size'].values
df['Size Cat'] = temp_df['Size Cat'].values
del temp_df

M                     2201704
Varies with device      74777
k                       36253
nan                       196
G                          13
Name: Size, dtype: int64


In [154]:
#date_of_dataset = 2021
# df['Time since last update'] = pd.to_datetime(df['Last Updated']).apply(lambda date: date_of_dataset - int(date.year))

In [155]:
df.isnull().sum()

App Name                  0
App Id                    0
Category                  0
Rating                22883
Rating Count          22883
Maximum Installs          0
Free                      0
Price                     0
Size                  74973
Released              71053
Last Updated              0
Ad Supported              0
In App Purchases          0
Super Category            0
Rating Bin          1082645
Name Length               0
Size Cat                  0
dtype: int64

In [156]:
df = df.dropna()

In [157]:
df.head()

,App Name,App Id,Category,Rating,Rating Count,Maximum Installs,Free,Price,Size,Released,Last Updated,Ad Supported,In App Purchases,Super Category,Rating Bin,Name Length,Size Cat
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,7662,True,0.0,2.9,"May 21, 2020","May 06, 2021",True,False,Other,Moderate,19,Known
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,19,True,0.0,1.8,"Sep 10, 2018","Oct 13, 2018",True,False,Personal & Social,Superb,50,Known
6,unlimited 4G data prank free app,getfreedata.superfatiza.unlimitedjiodataprank,Libraries & Demo,4.5,12.0,2567,True,0.0,2.5,"Sep 23, 2019","Sep 27, 2019",True,False,Other,Superb,32,Known
7,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2.0,39.0,702,True,0.0,16.0,"Jun 21, 2019","Jun 21, 2019",False,False,Personal & Social,Bad App Yo,21,Known
9,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,820.0,62433,True,0.0,3.5,"Sep 22, 2019","Oct 07, 2020",True,False,Personal & Social,Superb,32,Known


In [158]:
df['Last Updated'] = pd.to_datetime(df['Last Updated'])
df['Released'] = pd.to_datetime(df['Released'])
df['Days Release to Last Update'] = (df['Last Updated'] - df['Released']).dt.days

In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1165189 entries, 1 to 2312943
Data columns (total 18 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   App Name                     1165189 non-null  object        
 1   App Id                       1165189 non-null  object        
 2   Category                     1165189 non-null  object        
 3   Rating                       1165189 non-null  float64       
 4   Rating Count                 1165189 non-null  float64       
 5   Maximum Installs             1165189 non-null  int64         
 6   Free                         1165189 non-null  bool          
 7   Price                        1165189 non-null  float64       
 8   Size                         1165189 non-null  float64       
 9   Released                     1165189 non-null  datetime64[ns]
 10  Last Updated                 1165189 non-null  datetime64[ns]
 11  Ad Supporte

In [160]:
# minder dan 100.000 installs en minstens 10 ratings
df = df[(df['Maximum Installs'] < 100000) & (df['Rating Count'] >= 10)]

In [161]:
df['Rating Bin'].value_counts()

Superb        270606
Bad App Yo    253491
Moderate      242882
Name: Rating Bin, dtype: int64

**Code below is still in development**

In [162]:
def prepare_inputs(encoder_type, feature):
    try:
        ohe = encoder_type
        return df.join(pd.DataFrame(data=ohe.fit_transform(df[[feature]]).toarray(), dtype=np.int8, 
            columns=list(df[feature].unique())))
    except:
        print("Get right column")

# reset index to be able to perform the join
df = df.reset_index(drop=True)

# one hot encoder
df = prepare_inputs(sklearn.preprocessing.OneHotEncoder(), 'Rating Bin')
df = prepare_inputs(sklearn.preprocessing.OneHotEncoder(), 'Super Category')

In [163]:
df.columns

Index(['App Name', 'App Id', 'Category', 'Rating', 'Rating Count',
       'Maximum Installs', 'Free', 'Price', 'Size', 'Released', 'Last Updated',
       'Ad Supported', 'In App Purchases', 'Super Category', 'Rating Bin',
       'Name Length', 'Size Cat', 'Days Release to Last Update', 'Moderate',
       'Superb', 'Bad App Yo', 'Other', 'Personal & Social', 'Entertainment',
       'Development'],
      dtype='object')

In [164]:
df = df.drop(['Last Updated', 'Rating', 'Category', 'Super Category', 'Rating Bin', 'Other', 'Released'], axis=1)
# 'Rating Bin'

In [165]:
df.head()

,App Name,App Id,Rating Count,Maximum Installs,Free,Price,Size,Ad Supported,In App Purchases,Name Length,Size Cat,Days Release to Last Update,Moderate,Superb,Bad App Yo,Personal & Social,Entertainment,Development
0,Ampere Battery Info,com.webserveis.batteryinfo,64.0,7662,True,0.0,2.9,True,False,19,Known,350,0,1,0,0,1,0
1,unlimited 4G data prank free app,getfreedata.superfatiza.unlimitedjiodataprank,12.0,2567,True,0.0,2.5,True,False,32,Known,4,0,0,1,0,1,0
2,The Everyday Calendar,com.mozaix.simoneboard,39.0,702,True,0.0,16.0,False,False,21,Known,0,1,0,0,0,0,1
3,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,820.0,62433,True,0.0,3.5,True,False,32,Known,381,0,0,1,0,0,1
4,Dodge The Cars!,com.MrScratchEnterprises.CarDogeGame,55.0,329,True,0.0,51.0,False,False,15,Known,0,0,0,1,1,0,0


In [166]:
df['Rating Count'] = df['Rating Count'].astype('int')

In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766979 entries, 0 to 766978
Data columns (total 18 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   App Name                     766979 non-null  object 
 1   App Id                       766979 non-null  object 
 2   Rating Count                 766979 non-null  int32  
 3   Maximum Installs             766979 non-null  int64  
 4   Free                         766979 non-null  bool   
 5   Price                        766979 non-null  float64
 6   Size                         766979 non-null  float64
 7   Ad Supported                 766979 non-null  bool   
 8   In App Purchases             766979 non-null  bool   
 9   Name Length                  766979 non-null  int64  
 10  Size Cat                     766979 non-null  object 
 11  Days Release to Last Update  766979 non-null  int64  
 12  Moderate                     766979 non-null  int8   
 13 

In [168]:
df.to_parquet('data\Google-Playstore-Modified.parquet')